# Södermanland	
# Sthlm

![My Image](./Map-of-Sweden-showing-the-21-administrative-regions-adapted-from-SCB-2007.png)


In [ ]:
#importing modules
from math import *
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 21})
import scipy.stats as stats
from scipy.optimize import curve_fit,leastsq
from scipy.integrate import odeint, ode
from scipy.interpolate import interp1d
import numpy as np
import pandas
from scipy.interpolate import interp1d

# 1. DATA preprocessing

In [120]:
#Parameters
n=20
startvecka=10-4 #start week
Tmax=48 #length of prediction
pv=0.023 #fraction of infected that are admitted

rho=7*1/5.1 #latency period
gamma=7*1/5 #recovery rate

delay=3#2.5 #delay from infection to admission

In [121]:
file = pandas.read_excel('../data/statistik-covid19-inskrivna_v2.xlsx',sheet_name="Inskrivna i slutenvård")
data = [['Sthlm', 8,2389923], ['VGR', 9,1725881], ['Skåne', 10,1387650],['Östergötland',11,467095],['Jönköping',12,364750],['Södermanland',13,299101],['Uppsala',14,387628],['Gävleborg',15,287660],['Västmanland',16,277074],['Örebro',17,305726],['Norrbotten',18,249768],['Dalarna',19,287806],['Västernorrland',20,244855],['Värmland',21,282840],['Halland',22,336132],['Kalmar',23,245992],['Kronoberg',24,202163],['Västerbotten',25,273061],['Jämtland',26,130972],['Blekinge',27,159349],['Gotland',28,60106]] 
regioner = pandas.DataFrame(data, columns = ['region', 'index','population']) 
regionkod=['SE-AB','SE-O','SE-M','SE-E','SE-F','SE-D','SE-C','SE-X','SE-U','SE-T','SE-BD','SE-W','SE-Y','SE-S','SE-N','SE-H','SE-G','SE-AC','SE-Z','SE-K','SE-I']
regioner['Regionkod'] = regionkod
yta=np.array([6524,23800,10968,10559,8190,10437,5427,8504,6075,28029,18118,17519,5118,54665,97239,11165,21549,8424,2931,48935,3135]) #area of region
regioner['Area']=yta
ffall = 9*np.ones(21).astype(int) #week number when first case was recorded
regioner['Första fall'] = ffall
startvecka = ffall - 3

regioner.drop(regioner.index[[20]],inplace=True) #remove Gotland


In [185]:
print(startvecka)
regioner

[6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6]


,region,index,population,Regionkod,Area,Första fall
0,Sthlm,8,2389923,SE-AB,6524,9
1,VGR,9,1725881,SE-O,23800,9
2,Skåne,10,1387650,SE-M,10968,9
3,Östergötland,11,467095,SE-E,10559,9
4,Jönköping,12,364750,SE-F,8190,9
5,Södermanland,13,299101,SE-D,10437,9
6,Uppsala,14,387628,SE-C,5427,9
7,Gävleborg,15,287660,SE-X,8504,9
8,Västmanland,16,277074,SE-U,6075,9
9,Örebro,17,305726,SE-T,28029,9


In [186]:
regioner['density'] = regioner['population']/regioner['Area']

In [123]:
file

,"Nyinskrivna i slutenvård med covid-19, antal patienter per inskrivningsvecka och region",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88
0,Antal unika patienter som rapporterats till So...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Totalt antal inskrivna patienter,NaN,Månadsrapportering till patientregistret,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,vecka 10,NaN,vecka 11,NaN,vecka 12,NaN,vecka 13,...,vecka 48,NaN,vecka 49,NaN,vecka 50,NaN,vecka 51,NaN,vecka 52,NaN
5,NaN,Antal,%*,Antal,%**,Antal,%**,Antal,%**,Antal,...,Antal,%**,Antal,%**,Antal,%**,Antal,%**,Antal,%**
6,Totalt inrapporterat,35057,100,63,0.179707,169,0.482072,639,1.822746,1465,...,1667,4.755113,1568,4.472716,1608,4.586816,1251,3.568474,365,1.041162
7,Rapporterande region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Stockholm,13035,37.182303,37,0.283851,83,0.636747,335,2.570004,734,...,632,4.848485,548,4.204066,521,3.996931,347,2.662064,43,0.329881
9,Västra Götaland,5293,15.098269,9,0.170036,18,0.340072,56,1.058001,94,...,205,3.87304,259,4.893255,323,6.102399,328,6.196864,202,3.816361


In [124]:
from scipy.interpolate import interp1d

def getData(namn,ffall):
    temp=regioner.loc[regioner['region']==namn] #find the right row
    index=temp['index'].values #find the region index
    N=temp['population'].values #find population size of region
    N = int(N[0])
    R = file.iloc[index[0]] 
    R=R.replace(' ','X') #replace missing data with 0
    R=R.replace('X',0) 
    R=R.replace('IU',0) 
    
    ind = np.arange(3,len(R),2) 
    I = R.iloc[ind].values #extract admissions
    #print(I)
    #print(len(I))
    I=I[:-2] #remove last (incomplete) data point
    #print(I)
    padding = np.zeros(10-ffall+3) 
    I=np.insert(I,0,padding) 
    #print(I)
    veckor = np.arange(0,len(I),1) #create array of week numbers
    #print(veckor)
    
    return I,veckor,N


#Load Google mobility data
def loadMobility(namn,svecka):
    df = pandas.read_csv('../data/2020_SE_Region_Mobility_Report.csv') 
    temp=regioner.loc[regioner['region']==namn] 
    kod=temp['Regionkod'].values 
    temp2=df.loc[df['iso_3166_2_code']==kod[0]]
    transit=temp2['transit_stations_percent_change_from_baseline'].values 
    dagar=np.arange(46,46+len(transit))
    tv=np.ceil(dagar/7)
    
    
    gveckor=np.unique(tv)
    k=0
    Tr=np.zeros(int(max(gveckor)-min(gveckor))+1)

    for i in gveckor:
        ind=tv==i
        Tr[k]=np.mean(transit[ind])/100
        k=k+1
    for i in range(1,len(Tr)):
        if np.isnan(Tr[i]):
            Tr[i]=Tr[i-1]
    #print(Tr)
    init=0.2 #set base line travel
    Tr[0:2]=[init,init]
    #print(Tr)
    #print(gveckor)
    #print(gveckor-svecka)
    Trfkn = interp1d(gveckor-svecka,Tr,kind="linear",fill_value="extrapolate")
    #print(Tr)
    return Trfkn

In [125]:
def loadMobility_list(namn_list, svecka):
    
    Trfkn_dict = dict()
    for i in namn_list:
       Trfkn_dict[i] = loadMobility(i,svecka) 

    return Trfkn_dict


def getData_list(namn_list, ffall):
    
    getData_dict = dict()
    for i in namn_list:
        I,veckor,N = getData(i,ffall)
        getData_dict[i] =  {'I':I,'veckor':veckor, 'N':N}

    return getData_dict

# 2. Correlation Analysis

In [212]:
namn_list = regioner.region.tolist()
Trfkn_dict = loadMobility_list(namn_list, svecka=6) 
getData_dict = getData_list(namn_list, ffall = 9) 

In [213]:
region_name = ['Sthlm']
df = pd.DataFrame(getData_dict['Sthlm']['I'])/regioner[regioner.region =='Sthlm'].density.values

for j in getData_dict.keys():
    if j == 'Sthlm':
        continue
    region_name.append(j)
    df_temp = pd.DataFrame(getData_dict[j]['I'])
  
    df_temp = df_temp/regioner[regioner.region ==j].density.values
    df = pd.concat([df,df_temp], axis = 1)
 

In [219]:
df.columns = region_name
df.head()

,Sthlm,VGR,Skåne,Östergötland,Jönköping,Södermanland,Uppsala,Gävleborg,Västmanland,Örebro,Norrbotten,Dalarna,Västernorrland,Värmland,Halland,Kalmar,Kronoberg,Västerbotten,Jämtland,Blekinge
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.101002,0.124111,0.0,0.0,0.0,0.0,0.056002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [222]:
df_Sthlm = pd.DataFrame(df.corr().loc['Sthlm']).sort_values(by = 'Sthlm', ascending=False)
df_Sthlm

,Sthlm
Sthlm,1.000000
Östergötland,0.968330
Dalarna,0.942713
Västmanland,0.919852
Norrbotten,0.916373
Södermanland,0.912002
Jönköping,0.911792
Uppsala,0.906437
Gävleborg,0.851443
Kalmar,0.845498


In [223]:
df_Skåne= pd.DataFrame(df.corr().loc['Skåne']).sort_values(by = 'Skåne', ascending=False)
df_Skåne

,Skåne
Skåne,1.000000
Kronoberg,0.895035
VGR,0.887548
Västmanland,0.844795
Kalmar,0.837116
Uppsala,0.836551
Södermanland,0.826470
Västernorrland,0.825453
Gävleborg,0.770706
Jämtland,0.714445


In [224]:
df_Uppsala= pd.DataFrame(df.corr().loc['Uppsala']).sort_values(by = 'Uppsala', ascending=False)
df_Uppsala

,Uppsala
Uppsala,1.000000
Västmanland,0.951433
Södermanland,0.935397
Kalmar,0.910907
Sthlm,0.906437
VGR,0.903484
Västernorrland,0.879517
Östergötland,0.836592
Skåne,0.836551
Gävleborg,0.834167


# 3. Modeling

#### S[Sthlm]=x1      

#### E[Sthlm]=x2       

#### I[Sthlm]=x3       

#### R[Sthlm]=x4

#### S[Södermanland]=x5      

#### E[Södermanland]=x6       

#### I[Södermanland]=x7       

#### R[Södermanland]=x8

In [139]:
namn_list = ['Sthlm','Södermanland']
Trfkn_dict = loadMobility_list(namn_list, svecka=6) 
getData_dict = getData_list(namn_list, ffall = 9) 
region_1 = namn_list[0]
region_2 = namn_list[1] 

def beta(t,a,b,transit_fkn):
    return a+b*transit_fkn(t)

def dx(x,t,a1,b1, a2, b2, c, lag):
    x1, x2, x3, x4, x5, x6, x7, x8 = x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7]
    
    dx1_sthlm =  -x1 * (beta(t-lag,a1,b1,transit_fkn_dct[region_1])*x3/getData_dict[region_1]['N']  + c* x7/getData_dict[region_2]['N'])
    dx1_söd =  -x5 * (beta(t-lag,a2,b2,transit_fkn_dct[region_2])*x7/getData_dict[region_2]['N'] + c* x3/getData_dict[region_1]['N'])
    
    dx2_sthlm = x1 * (beta(t-lag,a1,b1,transit_fkn_dct[region_1])*x3/getData_dict[region_1]['N']  + c* x7/getData_dict[region_2]['N'])- rho*x2
    dx2_söd = x5 * (beta(t-lag,a2,b2,transit_fkn_dct[region_2])*x3/getData_dict[region_2]['N'] + c* x3/getData_dict[region_1]['N']) - rho*x6
    
    dx3_sthlm  = rho*x2 - gamma*x3
    dx3_söd = rho*x6 - gamma*x7
    
    dx4_sthlm  = gamma*x3
    dx4_söd = gamma*x7
    
    m = [dx1_sthlm, dx2_sthlm, dx3_sthlm, dx4_sthlm, dx1_söd, dx2_söd, dx3_söd, dx4_söd]

    return m     

def pred_inlagda(t,a1,b1, a2, b2, c,lag):
    N_region_1 = getData_dict[region_1]['N']
    N_region_2 = getData_dict[region_2]['N']
    
    x0 = [N_region_1-1,0,1,0, N_region_2-1,0,1,0] #initial conditions
 

    x = odeint(dx, x0, t,args=(a1,b1, a2, b2, c,lag))

    return pv*x[:,2], pv*x[:,6]

In [141]:
x0

[2389922, 0, 1, 0, 299100, 0, 1, 0]

In [143]:
namn_list = ['Sthlm','Södermanland']
Trfkn_dict = loadMobility_list(namn_list, svecka=6) 
getData_dict = getData_list(namn_list, ffall = 9) 
region_1 = namn_list[0]
region_2 = namn_list[1] 
getData_dict

{'Sthlm': {'I': array([0.0, 0.0, 0.0, 0.0, 37, 83, 335, 734, 968, 984, 872, 739, 567, 548,
         448, 394, 440, 364, 292, 220, 171, 111, 82, 57, 52, 41, 47, 38, 28,
         23, 29, 30, 31, 31, 57, 88, 78, 131, 262, 399, 531, 592, 632, 548,
         521], dtype=object),
  'veckor': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
         17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
         34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44]),
  'N': 2389923},
 'Södermanland': {'I': array([0.0, 0.0, 0.0, 0.0, 0, 9, 13, 54, 76, 53, 63, 53, 54, 54, 47, 42,
         53, 38, 45, 43, 33, 30, 14, 7, 12, 8, 13, 7, 0, 0, 4, 6, 0, 0, 0,
         15, 10, 16, 24, 44, 54, 50, 61, 50, 70], dtype=object),
  'veckor': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
         17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
         34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44]),
  'N': 299101}}

In [156]:
namn_list = regioner.region.tolist()
Trfkn_dict = loadMobility_list(namn_list, svecka=6) 
getData_dict = getData_list(namn_list, ffall = 9) 

In [159]:
getData_dict.keys()

dict_keys(['Sthlm', 'VGR', 'Skåne', 'Östergötland', 'Jönköping', 'Södermanland', 'Uppsala', 'Gävleborg', 'Västmanland', 'Örebro', 'Norrbotten', 'Dalarna', 'Västernorrland', 'Värmland', 'Halland', 'Kalmar', 'Kronoberg', 'Västerbotten', 'Jämtland', 'Blekinge'])

In [ ]:
df

In [191]:
regioner[regioner.region == 'Sthlm'].density.values

array([366.32786634])

In [202]:
region_name = ['Sthlm']
df = pd.DataFrame(getData_dict['Sthlm']['I'])/regioner[regioner.region =='Sthlm'].density.values

for j in getData_dict.keys():
    if j == 'Sthlm':
        continue
    region_name.append(j)
    df_temp = pd.DataFrame(getData_dict[j]['I'])
  
    df_temp = df_temp/regioner[regioner.region ==j].density.values
    df = pd.concat([df,df_temp], axis = 1)
 

[72.5160084]
[126.51805252]
[44.23667014]
[44.53601954]
[28.65775606]
[71.42583379]
[33.82643462]
[45.60888889]
[10.90748867]
[13.78562755]
[16.42822079]
[47.84193044]
[5.17406018]
[3.45676118]
[22.03242275]
[9.38154903]
[32.41464862]
[44.68509041]
[3.25634004]


In [203]:
df.columns = region_name


,Sthlm,VGR,Skåne,Östergötland,Jönköping,Södermanland,Uppsala,Gävleborg,Västmanland,Örebro,Norrbotten,Dalarna,Västernorrland,Värmland,Halland,Kalmar,Kronoberg,Västerbotten,Jämtland,Blekinge
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.101002,0.124111,0.0,0.0,0.0,0.0,0.056002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.226573,0.248221,0.0,0.226057,0.157176,0.314051,0.056002,0.295627,0.0,0.0,0.0,0.304354,0.0,0.773087,0.0,0.0,0.0,0.0,0.0,0.0
6,0.914481,0.772243,0.1976,0.972044,0.830788,0.453629,0.140005,0.591254,0.131553,1.466882,1.08809,1.034805,0.083609,1.546175,2.314305,0.408489,0.0,0.123401,0.268546,0.0
7,2.00367,1.296266,0.347776,2.780499,2.559726,1.884307,0.588023,0.975568,0.855096,2.750404,1.523326,3.469639,0.209022,1.159631,6.942915,0.453877,0.746145,0.215952,0.246167,1.228373
8,2.642442,2.054719,0.648129,4.136839,3.210884,2.651987,0.966037,1.359883,1.162054,3.483845,3.554427,3.226156,0.459848,0.966359,4.339322,0.907753,0.0,0.277652,0.58185,1.535466
9,2.686118,3.240664,0.766689,3.50388,2.627087,1.849412,0.924035,1.537259,1.512863,2.750404,4.642516,3.652252,0.731576,2.899077,5.496475,1.089304,1.705475,0.308502,0.514713,0.0


In [204]:
df.corr()

,Sthlm,VGR,Skåne,Östergötland,Jönköping,Södermanland,Uppsala,Gävleborg,Västmanland,Örebro,Norrbotten,Dalarna,Västernorrland,Värmland,Halland,Kalmar,Kronoberg,Västerbotten,Jämtland,Blekinge
Sthlm,1.000000,0.843206,0.707943,0.968330,0.911792,0.912002,0.906437,0.851443,0.919852,0.608364,0.916373,0.942713,0.767337,0.609362,0.797345,0.845498,0.634756,0.777421,0.834126,0.277613
VGR,0.843206,1.000000,0.887548,0.728929,0.583918,0.913086,0.903484,0.892073,0.902264,0.570380,0.880070,0.705631,0.886026,0.854733,0.793608,0.925145,0.855745,0.805349,0.664991,0.540573
Skåne,0.707943,0.887548,1.000000,0.621677,0.400992,0.826470,0.836551,0.770706,0.844795,0.357385,0.694584,0.537343,0.825453,0.701450,0.634010,0.837116,0.895035,0.708041,0.714445,0.703238
Östergötland,0.968330,0.728929,0.621677,1.000000,0.949446,0.843867,0.836592,0.758317,0.858380,0.558964,0.861616,0.937252,0.634197,0.456047,0.695051,0.749121,0.519094,0.666688,0.848179,0.214819
Jönköping,0.911792,0.583918,0.400992,0.949446,1.000000,0.742007,0.728714,0.634586,0.728252,0.546562,0.769218,0.934666,0.496379,0.328326,0.611534,0.626122,0.309441,0.592080,0.726492,0.029500
Södermanland,0.912002,0.913086,0.826470,0.843867,0.742007,1.000000,0.935397,0.890879,0.919812,0.666803,0.861836,0.819782,0.839591,0.734929,0.810330,0.888893,0.747598,0.818094,0.757545,0.496441
Uppsala,0.906437,0.903484,0.836551,0.836592,0.728714,0.935397,1.000000,0.834167,0.951433,0.563244,0.822067,0.805795,0.879517,0.724591,0.791194,0.910907,0.725122,0.833061,0.812731,0.496166
Gävleborg,0.851443,0.892073,0.770706,0.758317,0.634586,0.890879,0.834167,1.000000,0.888467,0.611241,0.894599,0.780226,0.822392,0.799760,0.880823,0.863245,0.827918,0.790691,0.698229,0.388709
Västmanland,0.919852,0.902264,0.844795,0.858380,0.728252,0.919812,0.951433,0.888467,1.000000,0.591124,0.874052,0.816632,0.876964,0.703475,0.832255,0.882670,0.768168,0.795805,0.850684,0.471452
Örebro,0.608364,0.570380,0.357385,0.558964,0.546562,0.666803,0.563244,0.611241,0.591124,1.000000,0.661141,0.610308,0.579358,0.611706,0.725711,0.523089,0.391109,0.514327,0.427815,0.262842


In [157]:
import pandas as pd

df = pd.DataFrame([getData_dict['Sthlm']['I'], getData_dict['Södermanland']['I']]).T
df = df.rename(columns = {0:'Sthlm',1:'Södermanland'})

In [158]:
df.corr()

,Sthlm,Södermanland
Sthlm,1.000000,0.912002
Södermanland,0.912002,1.000000


In [72]:
R = 'Sthlm'
k = 0
I,veckor,N = getData(R,ffall[k])
transit_fkn=loadMobility(R,startvecka[k])

I_train=I
veckor_train=veckor
n=10
ah=np.linspace(3,10,n)
bh=np.linspace(4,10,n)

E=np.zeros((len(ah),len(bh)))
for i in range(n):
    print(i)
    for j in range(n):
        P=pred_inlagda(veckor_train,ah[i],bh[j],delay)
        E[i,j]=np.sqrt(np.sum((P-I_train)**2)/len(veckor_train))

[-0.0025     -0.02       -0.05857143 -0.03428571 -0.25142857 -0.43714286
 -0.45142857 -0.47428571 -0.52142857 -0.42428571 -0.41142857 -0.45
 -0.39285714 -0.39571429 -0.38285714 -0.30571429 -0.32285714 -0.28571429
 -0.33285714 -0.27142857 -0.35       -0.36428571 -0.35142857 -0.35285714
 -0.33714286 -0.30285714 -0.27       -0.27       -0.28142857 -0.28857143
 -0.29428571 -0.26571429 -0.26285714 -0.29714286 -0.30285714 -0.32142857
 -0.31571429 -0.36428571 -0.39142857 -0.41571429 -0.444     ]
[ 0.2         0.2        -0.05857143 -0.03428571 -0.25142857 -0.43714286
 -0.45142857 -0.47428571 -0.52142857 -0.42428571 -0.41142857 -0.45
 -0.39285714 -0.39571429 -0.38285714 -0.30571429 -0.32285714 -0.28571429
 -0.33285714 -0.27142857 -0.35       -0.36428571 -0.35142857 -0.35285714
 -0.33714286 -0.30285714 -0.27       -0.27       -0.28142857 -0.28857143
 -0.29428571 -0.26571429 -0.26285714 -0.29714286 -0.30285714 -0.32142857
 -0.31571429 -0.36428571 -0.39142857 -0.41571429 -0.444     ]
0
1
2
3
4
5


In [73]:
ah

array([ 3.        ,  3.77777778,  4.55555556,  5.33333333,  6.11111111,
        6.88888889,  7.66666667,  8.44444444,  9.22222222, 10.        ])

In [74]:
optind = np.where(E == np.amin(E))
afit = ah[optind[0]]
bfit = bh[optind[1]]

print("Minimum RMSE: ",E[optind])
print("a=",afit,"b=",bfit)

Minimum RMSE:  [216.66060986]
a= [4.55555556] b= [8.66666667]


In [75]:
optind 

(array([2]), array([7]))

In [76]:
regs = pandas.read_pickle('../data/param_IC.pkl') #load initial guesses for a and b for each region
astart = regs['a'].values
bstart = regs['b'].values

In [77]:
regs

,region,index,population,Regionkod,a,b,E
0,Sthlm,8,2389923,SE-AB,4.759494,9.215190,175.235300
1,VGR,9,1725881,SE-O,3.848101,9.493671,90.821751
2,Skåne,10,1387650,SE-M,3.620253,8.240506,41.357155
3,Östergötland,11,467095,SE-E,3.734177,8.379747,17.383155
4,Jönköping,12,364750,SE-F,3.278481,10.050633,21.330193
5,Södermanland,13,299101,SE-D,3.734177,9.215190,15.531288
6,Uppsala,14,387628,SE-C,3.506329,6.430380,17.883654
7,Gävleborg,15,287660,SE-X,3.164557,10.746835,20.700678
8,Västmanland,16,277074,SE-U,3.506329,6.848101,8.616789
9,Örebro,17,305726,SE-T,3.392405,8.379747,12.290108
